In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os


In [2]:
def load_model(model_path):
    model = tf.keras.models.load_model(model_path, compile=False)
    return model

def preprocess_image(image_path, target_size=(128, 128)):
    original = cv2.imread(image_path)
    if original is None:
        raise ValueError(f"Image not found: {image_path}")
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(original_rgb, target_size)
    normalized = resized.astype("float32") / 255.0
    return original, normalized

def predict_keypoints(model, image_array, original_size):
    input_batch = np.expand_dims(image_array, axis=0)  # shape: (1, 128, 128, 3)
    preds = model.predict(input_batch)[0]  # shape: (10,)
    preds = preds.reshape(5, 2)  # 5 keypoints

    # Rescale to original image size
    h, w = original_size
    preds[:, 0] *= w  # x-coordinates
    preds[:, 1] *= h  # y-coordinates
    return preds

def draw_keypoints(image, keypoints, color=(0, 255, 0)):
    h, w = image.shape[:2]
    radius = max(2, int(0.005 * max(h, w)))  # ~0.5% of image dimension
    for (x, y) in keypoints.astype(int):
        cv2.circle(image, (x, y), radius=radius, color=color, thickness=-1)
    return image

def show_resized_window(window_name, image, max_dim=800):
    h, w = image.shape[:2]
    scale = min(max_dim / max(h, w), 1.0)
    resized = cv2.resize(image, (int(w * scale), int(h * scale)))
    cv2.imshow(window_name, resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def run_on_random_image(model_path, image_path):
    model = load_model(model_path)
    original_img, preprocessed_img = preprocess_image(image_path)
    original_size = original_img.shape[:2]  # (height, width)

    keypoints = predict_keypoints(model, preprocessed_img, original_size)
    print("🔍 Predicted Keypoints:\n", keypoints)

    vis_img = draw_keypoints(original_img.copy(), keypoints)
    show_resized_window("Predicted Keypoints", vis_img)


In [17]:
run_on_random_image("EfficientNetV2B0.h5", "handOnMouth.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🔍 Predicted Keypoints:
 [[202.57327 178.43765]
 [132.3237  141.12312]
 [250.72772 143.2703 ]
 [156.88736 208.70575]
 [234.19151 209.24107]]
